In [23]:
from tvb.simulator.lab import *
import tvb.interfaces.command.demos as demo
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as st
import itertools
from sklearn import manifold
from sklearn.preprocessing import StandardScaler
%pylab nbagg

Populating the interactive namespace from numpy and matplotlib


/Users/mb/.virtualenvs/tvb/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['mat']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [24]:
mat = scipy.io.loadmat('empirical_fMRI/empirical_fMRI.mat')
eeg = scipy.io.loadmat('HRF_200Hz.mat')

In [25]:
eeg.keys()

dict_keys(['__header__', '__version__', '__globals__', 'HRF_resEEG'])

In [26]:
data = np.asarray([x[0][0] for x in mat['fMRI'].T])
data[0].shape

(661, 68)

In [27]:
scaler = StandardScaler()
data = np.asarray([scaler.fit_transform(subj) for subj in data])
data.shape
rslt = np.empty((data[0].shape[1],data[0].shape[1]))
print(rslt.shape)

(68, 68)


In [28]:
def get_cofluct(subj):
    rslt = np.empty((subj.shape[1], subj.shape[1], subj.shape[0]))
    for i, j in itertools.combinations_with_replacement(list(range(subj.shape[1])), r=2):
        if i != j:
            rslt[i,j] = subj[:,i]*subj[:,j]
            rslt[i,j] = rslt[i,j]#/np.mean(rslt[i,j])
            #rslt[j,i] = rslt[i,j]
    return rslt

In [29]:
def get_fc(subj):
    rslt = np.empty((subj.shape[1], subj.shape[1]))
    for i, j in itertools.combinations_with_replacement(list(range(subj.shape[1])), r=2):
        if i != j:
            rslt[i,j] = np.sum(subj[:,i]*subj[:,j])/660
            rslt[i,j] = rslt[i,j]#/np.mean(rslt[i,j])
            #rslt[j,i] = rslt[i,j]
    return rslt
    

In [30]:
X = get_cofluct(data[1])

In [31]:
X_fc = get_fc(data[1])

In [32]:
#plt.imshow(np.mean(X, axis=2))

In [33]:
X2 = X.reshape((X.shape[0]**2, X.shape[2]))
X2.shape

(4624, 661)

In [34]:
#plt.figure()
#plt.hist(np.linalg.norm(X2, axis=0), histtype='step', bins=200)
#plt.show()

In [35]:
np.percentile(np.linalg.norm(X2, axis=0), [5, 95])

array([ 20.99888008, 111.39508966])

In [42]:
bottom = np.where(np.linalg.norm(X2, axis=0) <= 20.99888008, X2, 0)
bottom = bottom.reshape((68,68,661))
top = np.where(np.linalg.norm(X2, axis=0) > 111.39508966, X2, 0)
top = top.reshape((68,68,661))

In [43]:
top_fc = np.mean(top, axis=2)
bot_fc = np.mean(bottom, axis=2)

In [44]:
print(top_fc)

[[0.         0.11414125 0.08065393 ... 0.09052258 0.13464836 0.14874723]
 [0.         0.         0.09320954 ... 0.06732544 0.1584081  0.18999028]
 [0.         0.         0.         ... 0.06129256 0.07413327 0.09534797]
 ...
 [0.         0.         0.         ... 0.         0.10114064 0.10360112]
 [0.         0.         0.         ... 0.         0.         0.21056645]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [45]:
print(bot_fc)

[[ 0.          0.00589286  0.00338872 ... -0.00303549 -0.00031785
  -0.00298513]
 [ 0.          0.          0.00453051 ... -0.00086682  0.00821968
   0.00394922]
 [ 0.          0.          0.         ... -0.00778827 -0.00352007
  -0.00592995]
 ...
 [ 0.          0.          0.         ...  0.         -0.00140824
   0.00464385]
 [ 0.          0.          0.         ...  0.          0.
   0.00805382]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [46]:
np.corrcoef(np.triu(X_fc).ravel(), np.triu(bot_fc).ravel())[0,1]

0.4290141494849377

In [47]:
np.corrcoef(np.triu(X_fc).ravel(), np.triu(top_fc).ravel())[0,1]

0.8817248758110365